In [58]:
from pathlib import Path
import sys, importlib

# project root is the parent of the research folder
PROJECT_ROOT = Path.cwd().parent     # .../Chicken-Disease-Classification
SRC = PROJECT_ROOT / "src"

# ensure src is first on sys.path
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

# hard-reload the module in case it was cached
import Chicken_Disease_Classifier.constants as C
importlib.reload(C)

from Chicken_Disease_Classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
print(CONFIG_FILE_PATH, PARAMS_FILE_PATH)



config\config.yaml params.yaml


In [59]:
import yaml
from box import ConfigBox
from Chicken_Disease_Classifier import logger

def read_yaml(path_to_yaml):
    with open(path_to_yaml, "r", encoding="utf-8") as f:
        content = yaml.safe_load(f)
    logger.info(f"yaml file: {path_to_yaml} loaded successfully")
    if content is None:
        raise ValueError(f"YAML is empty: {path_to_yaml}")
    if not isinstance(content, dict):
        raise ValueError(f"YAML must be a mapping at top-level, got {type(content).__name__}: {path_to_yaml}")
    return ConfigBox(content)


In [2]:
%pwd

'c:\\Users\\amanbatra\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [30]:
%pwd

'c:\\Users\\amanbatra\\Chicken-Disease-Classification'

In [27]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [60]:
from Chicken_Disease_Classifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from Chicken_Disease_Classifier.utils.common import read_yaml, create_directories


In [61]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [ ]:
import os
import urllib.request as request
import zipfile
from Chicken_Disease_Classifier import logger
from Chicken_Disease_Classifier.utils.common import get_size

In [63]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [64]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-22 04:21:27,022: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-22 04:21:27,024: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-22 04:21:27,025: INFO: common: created directory at: artifacts]
[2025-10-22 04:21:27,027: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-22 04:21:31,924: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 87C3:411FD:B3BB8:17B486:68F80E6A
Accept-Ranges: bytes
Date: Tue, 21 Oct 2025 22:51:28 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830044-CCU
X-Cache: M